In [1]:
import re
import pandas as pd

In [2]:
content = ''
with open('./data/本草綱目.txt', 'r', encoding='utf8') as file:
    for line in file:
        content += line
finds = re.findall(r'    \n(?:(^\S.*)(?:\n    \n))?(^\S.*)\n((?:    .*\n?)+)(?=    \n?)', content, re.M)
df = pd.DataFrame(columns=['目錄', '標題', '內文'])

for find in finds:
    df_row = dict((key,'') for key in df.columns)
    df_row['目錄'] = '\n'.join([t.strip() for t in find[0].strip().split('\n')])
    df_row['標題'] = '\n'.join([t.strip() for t in find[1].strip().split('\n')])
    df_row['內文'] = '\n'.join([t.strip() for t in find[2].strip().split('\n')])
    df = df.append(df_row, ignore_index=True)

In [ ]:
df[df.iloc[:,0]==''].index

In [3]:
df.to_csv('./data/本草綱目.csv', encoding='utf-8-sig')

In [4]:
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        row['內文'] = re.sub(r'【 【主治】', '【主治】', row['內文'])
        row['內文'] = re.sub(r'主治 【附方', '主治】 【附方', row['內文'])
        row['內文'] = re.sub(r'附方 吐血', '附方】 吐血', row['內文'])
        row['內文'] = re.sub(r'【别名】', '【釋名】', row['內文'])
        row['內文'] = re.sub(r'【主冶】', '【主治】', row['內文'])
df.to_csv('./data/本草綱目_Ver1.csv', encoding='utf-8-sig')

In [13]:
string = '''塵芒入目∶大藕洗搗，綿裹，滴汁入目中，即出也。（《普濟方》）
藕   【釋名】藕絲菜（五、六月嫩時，採爲蔬【氣味】甘，平，無毒。
【主治】生食，主霍亂後虛渴煩悶不能食，解酒食毒（蘇頌）。
功與藕同（時珍）。解煩毒，下瘀血（汪穎）。
藕節  【氣味】澀，平，無毒。
大明曰∶冷。伏硫黃。
'''
re.sub(r'(?:(\w+ ?\w*)  (【(?:.*)))', r'\\x\1\\x\n', string)

'塵芒入目∶大藕洗搗，綿裹，滴汁入目中，即出也。（《普濟方》）\n\\x藕 \\x\n\n【主治】生食，主霍亂後虛渴煩悶不能食，解酒食毒（蘇頌）。\n功與藕同（時珍）。解煩毒，下瘀血（汪穎）。\n\\x藕節\\x\n\n大明曰∶冷。伏硫黃。\n'

In [ ]:
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if not re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        continue
    if re.search(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M):
        ns = re.sub(r'(?:(^\w{1,10})(?: {2,2}【.*))', r'\\x\1\\x\n', row['內文'])
        print(ns)
#     sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))|(?:(^\w{2,10})(?: {1,1}【.*))', row['內文'], re.M)
#     sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M)

In [ ]:
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if not re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        continue
#     if re.match(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M):
#         ns = re.sub(r'(?:(^\w{1,10})(?: {2,2}【.*))', r'\\x\1\\x\n', row['內文'], re.M)
#         print(ns)
#     sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))|(?:(^\w{2,10})(?: {1,1}【.*))', row['內文'], re.M)
    sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M)
#     if sites:
#         print(sites)


In [ ]:
new_df = pd.DataFrame(columns=['目錄', '標題', '釋名', '內文', '部位', '部位內文', '主治', '氣味', '附方'])
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if not re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        continue
    data_row = dict((k, '') for k in new_df.columns)
    data_row['目錄'] = row['目錄']
    data_row['標題'] = row['標題']
    data_row['內文'] = row['內文']
    if re.search(r'【釋名】(.*?)(?=【|(?<=.$))', data_row['內文'], re.S):
        data_row['釋名'] = re.search(r'【釋名】(.*?)(?=【|(?<=.$))', data_row['內文'], re.S).group(1)
    newContent = re.sub(r'(?:(^\w{1,10})(?: {2,2}【.*))', r'\\x\1\\x\n\2', row['內文'], re.M)
    sites = re.findall(r'(\\x.+?\\x.*?)(?=\\x|(?<=.$))', newContent, re.S)
    sites = sites if sites else [data_row['內文']]
    for s_content in sites:
        data_row['部位內文'] = s_content
        if re.search(r'\\x(.+?)\\x', s_content, re.S):
            data_row['部位'] = re.search(r'\\x(.+?)\\x', s_content, re.S).group(1)
        if re.search(r'【主治】(.*?)(?=【|(?<=.$))', s_content, re.S):
            data_row['主治'] = re.search(r'【主治】(.*?)(?=【|(?<=.$))', s_content, re.S).group(1)
        if re.search(r'【氣味】(.*?)(?=【|(?<=.$))', s_content, re.S):
            data_row['氣味'] = re.search(r'【氣味】(.*?)(?=【|(?<=.$))', s_content, re.S).group(1)
        if re.search(r'【附方】(.*?)(?=【|(?<=.$))', s_content, re.S):
            data_row['附方'] = re.search(r'【附方】(.*?)(?=【|(?<=.$))', s_content, re.S).group(1)
        new_df = new_df.append(data_row, ignore_index=True)


In [ ]:
new_df.to_csv('./data/本草綱目_Ver2.csv', encoding='utf-8-sig')

In [ ]:
df = new_df

In [ ]:
c_contents = df.loc[:,'附方']

In [ ]:
compounds = []
for c_content in c_contents:
    if pd.isnull(c_content):
        continue
    pattern = r'[。）》\n][\w ]*?( ?\w+[\w ，、]{1,10})∶'
    finds = re.findall(pattern, c_content, re.S)
    if finds:
        compounds.extend(finds)
#         print(finds)
print(len(set(compounds)))
# print(compounds)
[c for c in set(compounds) if len(c)>10 and all(x in c for x in ['、'])]

In [ ]:
# (?=(?=(?:' + compound_join + ')∶)|(?<=.$))
# r'''(?:(?<=(?=\w))|(?:（|《|》|）|，|。))\w*? ?(\w+[ ，\w]+)∶(.*?)
# (?=(?=(?:(?:(?<=(?=\w))|(?:（|《|》|）|，|。))\w*? ?(\w+[ ，\w]+)∶(.*?)))|(?<=.$))'''
compounds = []

for c_content in c_contents[9:20]:
    if pd.isnull(c_content):
        continue
    pattern = r'[。）》\n][\w ]*?( ?\w+[\w ，、]{1,10})∶(.*?)(?=(?:[。）》\n][\w ]*?(?: ?\w+[\w ，、]{1,10})∶)|(?<=.$))'
    
    finds = re.findall(pattern, c_content, re.M)
    if finds:
        compounds.extend(finds)
        print(finds, '\n')
        print(c_content, '\n')

print(len(set(compounds)))

In [ ]:
string = '''新三。目不得瞑∶乃陽氣盛不得入于陰，陰氣虛，故目不得瞑。治法飲以半夏湯，用流水千裏外者八升，揚之萬遍，取其清五升煮之，炊葦薪火，置秫米一升，半夏五合，徐炊令竭爲一升半，去滓飲汁一小杯，日三飲，以知爲度。詳半夏下。（《靈樞經》）汗後奔豚∶茯苓桂枝甘草大棗湯，治發汗後臍下悸，欲作奔豚者。茯苓一兩，炙甘草二錢半，桂枝三錢，大棗二枚。以甘瀾水二升，先煮茯苓，納諸藥煮服之，日三。（張仲景《金匱要略》）
服藥過劑煩悶∶東流水飲一、二升。（《肘後方》）'''

In [ ]:
# pattern = r'[。）》]((?: ?\w{0,10}[ ，、]?\n?){0,1}' \
#           r'(?: ?\w{0,2}[，、] \n){0,2}' \
#           r'(?:(?:\w+[，、]){1,3} ?\n?){0,3}\w{1,11})∶'

pattern = r'[。）》\n][\w ]*?( ?\w+[\w ，、]{1,10})∶(.*?)(?=(?:[。）》\n][\w ]*?(?: ?\w+[\w ，、]{1,10})∶)|(?<=.$))'

(re.findall(pattern, string, re.M))